Esta notebook contiene bloques de código útiles para realizar Q-learning en el entorno "Taxi"

In [ ]:
#!pip install gymnasium
#!pip install gymnasium[toy-text]
#!pip install scikit-learn
import numpy as np
from taxi_env_extended import TaxiEnvExtended

In [ ]:
env = TaxiEnvExtended()

Obtener la cantidad de estados y acciones

In [ ]:
actions = env.action_space.n
states = env.observation_space.n

Inicialización de la tabla Q

In [ ]:
Q = np.zeros((states, actions))
Q

Obtención de la acción a partir de la tabla Q

In [ ]:
def optimal_policy(state, Q):
    action = np.argmax(Q[state])
    return action

# es la acción que maximiza el valor de Q para un estado dado
# vamos a tener 500 observaciones, cada una con 6 acciones posibles 
# (norte, sur, este, oeste, recoger, dejar)
# para cada observación, vamos a tener el Q con las 6 acciones posibles
# entonces agarramos el máximo de las 6 acciones posibles para cada observación

Epsilon-Greedy Policy

In [ ]:
def epsilon_greedy_policy(state, Q, epsilon=0.1):
    explore = np.random.binomial(1, epsilon)
    if explore:
        action = env.action_space.sample()
        print('explore')
    # exploit
    else:
        action = np.argmax(Q[state])
        print('exploit')
        
    return action

# con cierta probabilidad exploramos y con cierta probabilidad explotamos
# si exploramos, tomamos una acción aleatoria
# si explotamos, tomamos la acción que maximiza el valor de Q para ese estado (lo mismo que optimal_policy)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

def imgRender():
    img = env.render()
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    #img_pil = Image.fromarray(img)
    #img_pil.save('images/image_{step}.png')


Ejemplo de episodio 

In [ ]:
# training function
import wandb
def train(env, Q, alpha=0.1, gamma=0.9, epsilon=0.1, episodes=10):
    for i in range(episodes):
        obs,_ = env.reset()
        print(obs)
        done = False
        total_reward = 0
        step_count = 0
        while not done:
            state = obs
            action = epsilon_greedy_policy(state, Q, epsilon)
            obs, reward, done, _, _ = env.step(action)
            total_reward += reward
            step_count += 1
            Q[state, action] = Q[state, action] + alpha * (reward + gamma * np.max(Q[obs]) - Q[state, action])
            print('->', state, action, reward, obs, done)

        wandb.log({"total_reward": total_reward, "avg_q_value": np.mean(Q)})
        
        print('episode', i)
        print('total_reward', total_reward)
        print('total_steps', step_count)

# para probar el agente se usa optimal_policy, el epsilon_greedy_policy es solo para entrenar
# el objetivo es primero entrenar y después probar.

# es bueno separar el código de Q-Learning en un archivo aparte y después importarlo para Taxi y péndulo
# porque una parte de la experimentación es cambiar parámetros y ver cómo cambia el comportamiento del agente
# ir guardando cada experimentación en un archivo aparte para después comparar resultados

In [ ]:
# test the agent
#obs,_ = env.reset()
#done = False
#while not done:
#    state = obs
#    action = optimal_policy(state, Q)
#    obs, reward, done, _, _ = env.step(action)
#    imgRender()
#    print('->', state, action, reward, obs, done)

In [ ]:
#!pip install wandb

In [ ]:
import wandb

wandb.login()

In [ ]:
# inicializar la corrida
wandb.init(
    # establecer el nombre del proyecto
    project="taxi",

    # establecer hiperparámetros
    config={
    "alpha": 0.3,
    "gamma": 0.9,
    "epsilon": 0.5,
    "epochs": 10000,
    }
)

# entrenar el agente
train(env, Q, wandb.config.alpha, wandb.config.gamma, wandb.config.epsilon, wandb.config.epochs)


# finalizar la corrida
wandb.finish()

In [ ]:
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression

# Guardar el modelo en un archivo .pkl
with open('taxi.pkl', 'wb') as archivo:
    pickle.dump(Q, archivo)

print("Modelo guardado en 'modelo.pkl'")


In [ ]:
# Cargar el modelo desde el archivo .pkl
with open('modelo.pkl', 'rb') as archivo:
    modelo_cargado = pickle.load(archivo)

# Usar el modelo cargado para hacer predicciones
obs,_ = env.reset()
done = False
while not done:
    state = obs
    action = optimal_policy(state, modelo_cargado)
    obs, reward, done, _, _ = env.step(action)
    imgRender()
    print('->', state, action, reward, obs, done)
